# Sistema de Recuperación de Información

Nombre: Marcela Cabrera

# Instalación de dependencias necesarias

In [10]:
!pip install ir-datasets sentence-transformers faiss-cpu nltk pandas numpy scikit-learn tqdm

import ir_datasets
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from tqdm import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Carga del corpus

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

100%|██████████| 1.56G/1.56G [01:16<00:00, 21.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/270


# 1. PREPROCESAMIENTO DE DATOS:
 En esta etapa vamos a:

Cargar el archivo JSON de arXiv que contiene metadata de papers científicos
Limpiar y normalizar los textos (títulos y abstracts)

Aplicar técnicas de NLP: tokenización, eliminación de stopwords, stemming

Preparar los datos en un formato estructurado para procesamiento posterior

In [5]:
class TextPreprocessor:
    """
    Clase para preprocesar texto científico.
    Aplica: normalización, tokenización, eliminación de stopwords y stemming.
    """
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()

    def preprocess(self, text):
        if not text or pd.isna(text):
            return ""

        # Normalización a minúsculas
        text = text.lower()

        # Tokenización
        tokens = word_tokenize(text)

        # Eliminación de stopwords y caracteres no alfanuméricos
        tokens = [token for token in tokens
                 if token.isalnum() and token not in self.stop_words]

        # Stemming (reducir palabras a su raíz)
        tokens = [self.stemmer.stem(token) for token in tokens]

        return ' '.join(tokens)

# Inicializar preprocesador
preprocessor = TextPreprocessor()
print("✓ Preprocesador inicializado")

✓ Preprocesador inicializado


In [8]:
import json
# Cargar el dataset de arXiv
print("="*80)
print("CARGANDO DATASET ARXIV")
print("="*80)

# Ruta al archivo JSON descargado
dataset_path = "/root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/270"
json_file = os.path.join(dataset_path, "arxiv-metadata-oai-snapshot.json")

print(f"\nArchivo: {json_file}")
print(f"Tamaño del archivo: {os.path.getsize(json_file) / (1024**3):.2f} GB")

def load_arxiv_papers(filepath, max_papers=30000, categories_filter=None):
    """
    Carga papers de arXiv desde el archivo JSON.

    Args:
        filepath: ruta al archivo JSON
        max_papers: número máximo de papers a cargar
        categories_filter: lista de categorías a filtrar (ej: ['cs.AI', 'cs.LG'])
    """
    papers = []

    print(f"\nCargando hasta {max_papers:,} papers...")
    if categories_filter:
        print(f"Filtrando por categorías: {categories_filter}")

    with open(filepath, 'r', encoding='utf-8') as f:
        for i, line in enumerate(tqdm(f, desc="Procesando")):
            if len(papers) >= max_papers:
                break

            try:
                paper = json.loads(line)

                # Filtrar por categoría si se especifica
                if categories_filter:
                    paper_cats = paper.get('categories', '').split()
                    if not any(cat in categories_filter for cat in paper_cats):
                        continue

                papers.append({
                    'id': paper.get('id', ''),
                    'title': paper.get('title', ''),
                    'abstract': paper.get('abstract', ''),
                    'authors': paper.get('authors', ''),
                    'categories': paper.get('categories', ''),
                    'submitter': paper.get('submitter', ''),
                    'doi': paper.get('doi', ''),
                    'journal-ref': paper.get('journal-ref', ''),
                    'comments': paper.get('comments', ''),
                    'update_date': paper.get('update_date', '')
                })

            except (json.JSONDecodeError, Exception) as e:
                continue

    return pd.DataFrame(papers)

# Cargar papers (puedes filtrar por categorías de interés)
# Ejemplo: solo Computer Science y Machine Learning
cs_categories = ['cs.AI', 'cs.LG', 'cs.CL', 'cs.CV', 'cs.IR', 'cs.NE']

docs_df = load_arxiv_papers(
    json_file,
    max_papers=30000,  # Ajusta según tu RAM disponible
    categories_filter=None  # None = todas las categorías, o usa cs_categories
)

print(f"\n✓ Papers cargados: {len(docs_df):,}")

CARGANDO DATASET ARXIV

Archivo: /root/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/270/arxiv-metadata-oai-snapshot.json
Tamaño del archivo: 4.72 GB

Cargando hasta 30,000 papers...


Procesando: 30000it [00:00, 70783.13it/s]


✓ Papers cargados: 30,000


In [11]:
# Limpiar y preprocesar los textos
print("\n" + "="*80)
print("PREPROCESAMIENTO DE TEXTOS")
print("="*80)

# Limpiar saltos de línea y espacios extra en títulos y abstracts
print("\n1. Limpiando formato...")
docs_df['title_clean'] = docs_df['title'].str.replace('\n', ' ').str.replace('  ', ' ').str.strip()
docs_df['abstract_clean'] = docs_df['abstract'].str.replace('\n', ' ').str.replace('  ', ' ').str.strip()

# Crear texto completo (título + abstract)
print("2. Combinando título y abstract...")
docs_df['full_text'] = docs_df['title_clean'] + '. ' + docs_df['abstract_clean']

# Aplicar preprocesamiento (tokenización, stemming, etc.)
print("3. Aplicando tokenización, stemming y eliminación de stopwords...")
tqdm.pandas(desc="Preprocesando")
docs_df['preprocessed'] = docs_df['full_text'].progress_apply(preprocessor.preprocess)

# Eliminar papers sin abstract
docs_df = docs_df[docs_df['abstract_clean'].str.len() > 50].reset_index(drop=True)

print(f"\n✓ Papers procesados: {len(docs_df):,}")
print(f"\nEjemplo de paper preprocesado:")
print("-" * 80)
print(f"ID: {docs_df.iloc[0]['id']}")
print(f"Título original: {docs_df.iloc[0]['title_clean'][:100]}...")
print(f"Abstract original: {docs_df.iloc[0]['abstract_clean'][:150]}...")
print(f"Preprocesado: {docs_df.iloc[0]['preprocessed'][:150]}...")


PREPROCESAMIENTO DE TEXTOS

1. Limpiando formato...
2. Combinando título y abstract...
3. Aplicando tokenización, stemming y eliminación de stopwords...


Preprocesando: 100%|██████████| 30000/30000 [01:01<00:00, 489.71it/s]



✓ Papers procesados: 29,957

Ejemplo de paper preprocesado:
--------------------------------------------------------------------------------
ID: 0704.0001
Título original: Calculation of prompt diphoton production cross sections at Tevatron and  LHC energies...
Abstract original: A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. A...
Preprocesado: calcul prompt diphoton product cross section tevatron lhc energi fulli differenti calcul perturb quantum chromodynam present product massiv photon pai...


## 2. CREACIÓN DE CONSULTAS Y QRELS
Como arXiv no tiene consultas predefinidas ni juicios de relevancia (qrels), vamos a:

Crear consultas de ejemplo relevantes para diferentes áreas científicas

Generar qrels automáticos basados en similitud de categorías y keywords


In [12]:
print("\n" + "="*80)
print("CREACIÓN DE CONSULTAS")
print("="*80)

# Definir consultas representativas de diferentes dominios científicos
queries_list = [
    {
        'query_id': 'Q001',
        'title': 'Deep learning convolutional neural networks',
        'description': 'Papers about deep learning using convolutional neural networks for image processing and computer vision',
        'narrative': 'Find research papers discussing CNN architectures, deep learning models for visual recognition, image classification, or object detection using neural networks'
    },
    {
        'query_id': 'Q002',
        'title': 'Natural language processing transformer models',
        'description': 'Research on transformer-based models for natural language understanding',
        'narrative': 'Find papers about transformer architectures, attention mechanisms, BERT, GPT, or other language models for NLP tasks'
    },
    {
        'query_id': 'Q003',
        'title': 'Reinforcement learning algorithms',
        'description': 'Papers on reinforcement learning methods and applications',
        'narrative': 'Find research on RL algorithms, policy gradient methods, Q-learning, deep reinforcement learning, or multi-agent systems'
    },
    {
        'query_id': 'Q004',
        'title': 'Graph neural networks',
        'description': 'Graph-based deep learning and neural network architectures',
        'narrative': 'Find papers about GNN architectures, graph convolutional networks, message passing, or graph representation learning'
    },
    {
        'query_id': 'Q005',
        'title': 'Quantum computing algorithms',
        'description': 'Quantum algorithms and quantum information processing',
        'narrative': 'Find research on quantum algorithms, quantum gates, quantum circuits, quantum error correction, or quantum machine learning'
    },
    {
        'query_id': 'Q006',
        'title': 'Generative adversarial networks',
        'description': 'GAN architectures and applications for generative modeling',
        'narrative': 'Find papers about GAN training, generative models, image synthesis, or adversarial learning techniques'
    },
    {
        'query_id': 'Q007',
        'title': 'Transfer learning domain adaptation',
        'description': 'Transfer learning methods and domain adaptation techniques',
        'narrative': 'Find research on transfer learning, domain adaptation, few-shot learning, or meta-learning approaches'
    },
    {
        'query_id': 'Q008',
        'title': 'Time series forecasting',
        'description': 'Methods for time series prediction and forecasting',
        'narrative': 'Find papers about time series analysis, LSTM networks, temporal modeling, or sequence prediction methods'
    },
    {
        'query_id': 'Q009',
        'title': 'Explainable artificial intelligence',
        'description': 'Interpretability and explainability in machine learning models',
        'narrative': 'Find research on model interpretability, explainable AI, feature importance, or attention visualization'
    },
    {
        'query_id': 'Q010',
        'title': 'Federated learning privacy',
        'description': 'Federated learning and privacy-preserving machine learning',
        'narrative': 'Find papers about federated learning, differential privacy, secure multi-party computation, or privacy-preserving AI'
    }
]

queries_df = pd.DataFrame(queries_list)

# Crear texto completo de la consulta
queries_df['full_text'] = (queries_df['title'] + '. ' +
                           queries_df['description'] + '. ' +
                           queries_df['narrative'])

# Preprocesar consultas
queries_df['preprocessed'] = queries_df['full_text'].apply(preprocessor.preprocess)

print(f"✓ Consultas creadas: {len(queries_df)}")
print("\nConsultas:")
for idx, row in queries_df.iterrows():
    print(f"  {row['query_id']}: {row['title']}")


CREACIÓN DE CONSULTAS
✓ Consultas creadas: 10

Consultas:
  Q001: Deep learning convolutional neural networks
  Q002: Natural language processing transformer models
  Q003: Reinforcement learning algorithms
  Q004: Graph neural networks
  Q005: Quantum computing algorithms
  Q006: Generative adversarial networks
  Q007: Transfer learning domain adaptation
  Q008: Time series forecasting
  Q009: Explainable artificial intelligence
  Q010: Federated learning privacy


In [13]:
print("\n" + "="*80)
print("GENERACIÓN DE QRELS (Juicios de Relevancia)")
print("="*80)

def create_qrels_for_arxiv(queries_df, docs_df):
    """
    Crea qrels automáticos basados en:
    1. Coincidencia de keywords en título y abstract
    2. Similitud de categorías
    3. Scoring basado en múltiples criterios
    """

    # Definir keywords por cada consulta
    query_keywords = {
        'Q001': {
            'primary': ['deep learning', 'cnn', 'convolutional', 'neural network'],
            'secondary': ['image', 'vision', 'classification', 'recognition'],
            'categories': ['cs.CV', 'cs.LG', 'cs.AI']
        },
        'Q002': {
            'primary': ['transformer', 'bert', 'gpt', 'attention'],
            'secondary': ['nlp', 'language model', 'natural language'],
            'categories': ['cs.CL', 'cs.LG', 'cs.AI']
        },
        'Q003': {
            'primary': ['reinforcement learning', 'rl', 'policy gradient', 'q-learning'],
            'secondary': ['agent', 'reward', 'markov'],
            'categories': ['cs.LG', 'cs.AI', 'stat.ML']
        },
        'Q004': {
            'primary': ['graph neural', 'gnn', 'gcn', 'graph convolutional'],
            'secondary': ['node', 'edge', 'graph representation'],
            'categories': ['cs.LG', 'cs.AI', 'stat.ML']
        },
        'Q005': {
            'primary': ['quantum', 'qubit', 'quantum computing'],
            'secondary': ['quantum algorithm', 'quantum gate', 'quantum circuit'],
            'categories': ['quant-ph', 'cs.ET']
        },
        'Q006': {
            'primary': ['gan', 'generative adversarial', 'adversarial network'],
            'secondary': ['generator', 'discriminator', 'generative model'],
            'categories': ['cs.LG', 'cs.CV', 'stat.ML']
        },
        'Q007': {
            'primary': ['transfer learning', 'domain adaptation', 'few-shot'],
            'secondary': ['meta-learning', 'adaptation', 'fine-tuning'],
            'categories': ['cs.LG', 'cs.AI', 'stat.ML']
        },
        'Q008': {
            'primary': ['time series', 'forecasting', 'temporal'],
            'secondary': ['lstm', 'sequence', 'prediction'],
            'categories': ['cs.LG', 'stat.ML', 'cs.AI']
        },
        'Q009': {
            'primary': ['explainability', 'interpretability', 'explainable ai'],
            'secondary': ['attention', 'visualization', 'feature importance'],
            'categories': ['cs.LG', 'cs.AI', 'stat.ML']
        },
        'Q010': {
            'primary': ['federated learning', 'privacy', 'differential privacy'],
            'secondary': ['distributed', 'secure', 'privacy-preserving'],
            'categories': ['cs.LG', 'cs.CR', 'cs.AI']
        }
    }

    qrels_list = []

    print("Generando qrels para cada consulta...")
    for query_id, keywords_info in tqdm(query_keywords.items()):
        primary_kw = keywords_info['primary']
        secondary_kw = keywords_info['secondary']
        relevant_cats = keywords_info['categories']

        for idx, doc in docs_df.iterrows():
            text_lower = (doc['title_clean'] + ' ' + doc['abstract_clean']).lower()
            doc_categories = doc['categories'].split()

            # Scoring
            score = 0

            # Primary keywords en título (peso alto)
            for kw in primary_kw:
                if kw in doc['title_clean'].lower():
                    score += 3
                elif kw in text_lower:
                    score += 2

            # Secondary keywords
            for kw in secondary_kw:
                if kw in text_lower:
                    score += 1

            # Categorías relevantes
            for cat in relevant_cats:
                if cat in doc_categories:
                    score += 2

            # Asignar relevancia basada en score
            if score >= 8:
                relevance = 2  # Altamente relevante
            elif score >= 4:
                relevance = 1  # Relevante
            else:
                continue  # No relevante

            qrels_list.append({
                'query_id': query_id,
                'doc_id': doc['id'],
                'relevance': relevance,
                'score': score
            })

    return pd.DataFrame(qrels_list)

# Generar qrels
print("\nGenerando qrels basados en keywords y categorías...")
qrels_df = create_qrels_for_arxiv(queries_df, docs_df)

print(f"\n✓ Qrels generados: {len(qrels_df):,}")
print(f"\nEstadísticas de qrels:")
print(f"  Consultas con qrels: {qrels_df['query_id'].nunique()}")
print(f"  Promedio de docs relevantes por query: {len(qrels_df) / qrels_df['query_id'].nunique():.1f}")

print(f"\nDistribución de relevancia:")
print(qrels_df['relevance'].value_counts().sort_index())

print(f"\nQrels por consulta:")
for qid in queries_df['query_id']:
    count = len(qrels_df[qrels_df['query_id'] == qid])
    print(f"  {qid}: {count} documentos relevantes")


GENERACIÓN DE QRELS (Juicios de Relevancia)

Generando qrels basados en keywords y categorías...
Generando qrels para cada consulta...


100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


✓ Qrels generados: 1,743

Estadísticas de qrels:
  Consultas con qrels: 10
  Promedio de docs relevantes por query: 174.3

Distribución de relevancia:
relevance
1    1660
2      83
Name: count, dtype: int64

Qrels por consulta:
  Q001: 50 documentos relevantes
  Q002: 41 documentos relevantes
  Q003: 66 documentos relevantes
  Q004: 18 documentos relevantes
  Q005: 1449 documentos relevantes
  Q006: 13 documentos relevantes
  Q007: 18 documentos relevantes
  Q008: 52 documentos relevantes
  Q009: 20 documentos relevantes
  Q010: 16 documentos relevantes


# 3. REPRESENTACIÓN MEDIANTE EMBEDDINGS

 Vamos a convertir textos a representaciones vectoriales (embeddings):


Los embeddings capturan el significado semántico del texto

Esto permite buscar por similitud conceptual, no solo por palabras exactas

In [14]:
print("\n" + "="*80)
print("GENERACIÓN DE EMBEDDINGS")
print("="*80)

# Cargar modelo de embeddings especializado en papers científicos
print("\nCargando modelo de embeddings...")
print("Modelo: allenai-specter (optimizado para literatura científica)")

try:
    embedding_model = SentenceTransformer('allenai-specter')
    print("✓ Modelo SPECTER cargado")
except:
    print("⚠ SPECTER no disponible, usando all-MiniLM-L6-v2")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"Dimensión de embeddings: {embedding_model.get_sentence_embedding_dimension()}")


GENERACIÓN DE EMBEDDINGS

Cargando modelo de embeddings...
Modelo: allenai-specter (optimizado para literatura científica)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Modelo SPECTER cargado
Dimensión de embeddings: 768


In [15]:
# Generar embeddings para documentos
print("\n" + "="*80)
print("EMBEDDINGS DE DOCUMENTOS")
print("="*80)

batch_size = 32
doc_embeddings = []

print(f"Generando embeddings para {len(docs_df):,} documentos...")
print(f"Batch size: {batch_size}")

for i in tqdm(range(0, len(docs_df), batch_size), desc="Procesando batches"):
    batch_texts = docs_df['full_text'].iloc[i:i+batch_size].tolist()

    # Truncar textos muy largos (límite del modelo)
    batch_texts = [text[:512] if len(text) > 512 else text for text in batch_texts]

    batch_embeddings = embedding_model.encode(
        batch_texts,
        show_progress_bar=False,
        convert_to_numpy=True
    )
    doc_embeddings.extend(batch_embeddings)

doc_embeddings = np.array(doc_embeddings).astype('float32')

print(f"\n✓ Embeddings de documentos generados")
print(f"  Shape: {doc_embeddings.shape}")
print(f"  Tamaño en memoria: {doc_embeddings.nbytes / (1024**2):.2f} MB")


EMBEDDINGS DE DOCUMENTOS
Generando embeddings para 29,957 documentos...
Batch size: 32


Procesando batches: 100%|██████████| 937/937 [04:54<00:00,  3.18it/s]



✓ Embeddings de documentos generados
  Shape: (29957, 768)
  Tamaño en memoria: 87.76 MB


In [17]:
# Generar embeddings para consultas
print("\n" + "="*80)
print("EMBEDDINGS DE CONSULTAS")
print("="*80)

print(f"Generando embeddings para {len(queries_df)} consultas...")

query_embeddings = embedding_model.encode(
    queries_df['full_text'].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True
)
query_embeddings = np.array(query_embeddings).astype('float32')

print(f"\n✓ Embeddings de consultas generados")
print(f"  Shape: {query_embeddings.shape}")
print(f"  Tamaño en memoria: {query_embeddings.nbytes / 1024:.2f} KB")


EMBEDDINGS DE CONSULTAS
Generando embeddings para 10 consultas...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✓ Embeddings de consultas generados
  Shape: (10, 768)
  Tamaño en memoria: 30.00 KB


# 4. RECUPERACIÓN INICIAL CON FAISS


FAISS (Facebook AI Similarity Search) permite buscar en millones de vectores rápidamente

Usaremos producto interno (Inner Product) como medida de similitud

Recuperaremos los top-100 documentos más similares para cada consulta

In [18]:
print("\n" + "="*80)
print("CREACIÓN DE ÍNDICE FAISS")
print("="*80)

# Obtener dimensión de los embeddings
dimension = doc_embeddings.shape[1]
print(f"Dimensión de vectores: {dimension}")

# Crear índice FAISS con Inner Product
print("\nCreando índice FAISS (IndexFlatIP)...")
index = faiss.IndexFlatIP(dimension)

print("Normalizando embeddings para similitud coseno...")
# Normalizar vectores L2 para usar Inner Product como similitud coseno
faiss.normalize_L2(doc_embeddings)
faiss.normalize_L2(query_embeddings)

# Añadir documentos al índice
print("Indexando documentos...")
index.add(doc_embeddings)

print(f"\n✓ Índice FAISS creado exitosamente")
print(f"  Total de vectores indexados: {index.ntotal:,}")
print(f"  Tipo de índice: {type(index).__name__}")
print(f"  Es entrenado: {index.is_trained}")


CREACIÓN DE ÍNDICE FAISS
Dimensión de vectores: 768

Creando índice FAISS (IndexFlatIP)...
Normalizando embeddings para similitud coseno...
Indexando documentos...

✓ Índice FAISS creado exitosamente
  Total de vectores indexados: 29,957
  Tipo de índice: IndexFlatIP
  Es entrenado: True


In [19]:
print("\n" + "="*80)
print("RECUPERACIÓN INICIAL (First-Stage Retrieval)")
print("="*80)

k_initial = 100  # Top-k documentos a recuperar
initial_results = {}

print(f"Recuperando top-{k_initial} documentos para cada consulta...")

for idx, row in tqdm(queries_df.iterrows(), total=len(queries_df), desc="Consultas"):
    query_id = row['query_id']
    query_emb = query_embeddings[idx].reshape(1, -1)

    # Buscar en el índice
    scores, indices = index.search(query_emb, k_initial)

    # Guardar resultados
    initial_results[query_id] = {
        'doc_ids': [docs_df.iloc[i]['id'] for i in indices[0]],
        'doc_indices': indices[0].tolist(),
        'scores': scores[0].tolist()
    }

print(f"\n✓ Recuperación inicial completada")
print(f"  Consultas procesadas: {len(initial_results)}")
print(f"  Documentos por consulta: {k_initial}")
print(f"  Total de recuperaciones: {len(initial_results) * k_initial:,}")


RECUPERACIÓN INICIAL (First-Stage Retrieval)
Recuperando top-100 documentos para cada consulta...


Consultas: 100%|██████████| 10/10 [00:00<00:00, 98.97it/s]


✓ Recuperación inicial completada
  Consultas procesadas: 10
  Documentos por consulta: 100
  Total de recuperaciones: 1,000


# 5. RE-RANKING DE RESULTADOS
Mejoraremos los resultados iniciales con re-ranking:

Los Cross-Encoders analizan la relación entre query y documento de forma más profunda
Son más lentos pero más precisos que los embeddings


In [20]:
print("\n" + "="*80)
print("CARGA DE MODELO DE RE-RANKING")
print("="*80)

print("Cargando Cross-Encoder para re-ranking...")
print("Modelo: cross-encoder/ms-marco-MiniLM-L-6-v2")

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

print("✓ Modelo de re-ranking cargado")


CARGA DE MODELO DE RE-RANKING
Cargando Cross-Encoder para re-ranking...
Modelo: cross-encoder/ms-marco-MiniLM-L-6-v2


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✓ Modelo de re-ranking cargado


In [21]:
print("\n" + "="*80)
print("RE-RANKING DE RESULTADOS")
print("="*80)

top_k_final = 20  # Top-k documentos finales
final_results = {}

print(f"Re-rankeando top-{k_initial} → top-{top_k_final} para cada consulta...")

for idx, row in tqdm(queries_df.iterrows(), total=len(queries_df), desc="Re-ranking"):
    query_id = row['query_id']
    query_text = row['full_text']

    # Obtener documentos iniciales
    initial_doc_ids = initial_results[query_id]['doc_ids']

    # Preparar textos de documentos (título + abstract truncado)
    doc_texts = []
    valid_doc_ids = []

    for doc_id in initial_doc_ids:
        doc_row = docs_df[docs_df['id'] == doc_id]
        if len(doc_row) > 0:
            # Truncar a 512 caracteres para el cross-encoder
            text = doc_row.iloc[0]['full_text'][:512]
            doc_texts.append(text)
            valid_doc_ids.append(doc_id)

    # Crear pares [query, documento]
    pairs = [[query_text[:512], doc_text] for doc_text in doc_texts]

    # Obtener scores del cross-encoder
    scores = reranker.predict(pairs, show_progress_bar=False)

    # Ordenar por score descendente
    ranked_indices = np.argsort(scores)[::-1][:top_k_final]

    # Guardar resultados re-rankeados
    final_results[query_id] = {
        'doc_ids': [valid_doc_ids[i] for i in ranked_indices],
        'scores': [float(scores[i]) for i in ranked_indices]
    }

print(f"\n✓ Re-ranking completado")
print(f"  Consultas procesadas: {len(final_results)}")
print(f"  Documentos finales por consulta: {top_k_final}")


RE-RANKING DE RESULTADOS
Re-rankeando top-100 → top-20 para cada consulta...


Re-ranking: 100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


✓ Re-ranking completado
  Consultas procesadas: 10
  Documentos finales por consulta: 20


# 6. SIMULACIÓN Y VISUALIZACIÓN DE CONSULTAS
Mostraremos los resultados obtenidos:

Compararemos resultados antes y después del re-ranking

Mostraremos información completa de los papers recuperados

Analizaremos la mejora en relevancia con los qrels

In [22]:
def display_results(query_id, n_show=5):
    """
    Muestra resultados de una consulta antes y después del re-ranking
    """
    # Información de la consulta
    query_info = queries_df[queries_df['query_id'] == query_id].iloc[0]

    print(f"\n{'='*100}")
    print(f"CONSULTA: {query_id}")
    print(f"{'='*100}")
    print(f"Título: {query_info['title']}")
    print(f"Descripción: {query_info['description']}")
    print(f"{'='*100}")

    # RESULTADOS INICIALES
    print(f"\n{'─'*100}")
    print(f"RESULTADOS INICIALES (FAISS) - Top {n_show}")
    print(f"{'─'*100}")

    initial_doc_ids = initial_results[query_id]['doc_ids'][:n_show]
    initial_scores = initial_results[query_id]['scores'][:n_show]

    for i, (doc_id, score) in enumerate(zip(initial_doc_ids, initial_scores), 1):
        doc = docs_df[docs_df['id'] == doc_id].iloc[0]

        print(f"\n{i}. [{doc_id}] Score: {score:.4f}")
        print(f"   Título: {doc['title_clean']}")
        print(f"   Categorías: {doc['categories']}")
        print(f"   Abstract: {doc['abstract_clean'][:200]}...")
        print(f"   URL: https://arxiv.org/abs/{doc_id}")

    # RESULTADOS FINALES (RE-RANKING)
    print(f"\n{'─'*100}")
    print(f"RESULTADOS FINALES (Re-ranking) - Top {n_show}")
    print(f"{'─'*100}")

    final_doc_ids = final_results[query_id]['doc_ids'][:n_show]
    final_scores = final_results[query_id]['scores'][:n_show]

    for i, (doc_id, score) in enumerate(zip(final_doc_ids, final_scores), 1):
        doc = docs_df[docs_df['id'] == doc_id].iloc[0]

        print(f"\n{i}. [{doc_id}] Score: {score:.4f}")
        print(f"   Título: {doc['title_clean']}")
        print(f"   Categorías: {doc['categories']}")
        print(f"   Abstract: {doc['abstract_clean'][:200]}...")
        print(f"   URL: https://arxiv.org/abs/{doc_id}")

    # ANÁLISIS DE RELEVANCIA
    print(f"\n{'─'*100}")
    print(f"ANÁLISIS DE RELEVANCIA (basado en qrels)")
    print(f"{'─'*100}")

    # Obtener documentos relevantes según qrels
    relevant_docs = qrels_df[
        (qrels_df['query_id'] == query_id) &
        (qrels_df['relevance'] > 0)
    ]['doc_id'].tolist()

    initial_top = set(initial_results[query_id]['doc_ids'][:n_show])
    final_top = set(final_results[query_id]['doc_ids'][:n_show])

    initial_relevant = len(initial_top & set(relevant_docs))
    final_relevant = len(final_top & set(relevant_docs))

    print(f"Total de documentos relevantes: {len(relevant_docs)}")
    print(f"Relevantes en top-{n_show} inicial: {initial_relevant} ({initial_relevant/n_show*100:.1f}%)")
    print(f"Relevantes en top-{n_show} final: {final_relevant} ({final_relevant/n_show*100:.1f}%)")

    if final_relevant > initial_relevant:
        print(f"✓ Mejora: +{final_relevant - initial_relevant} documentos relevantes")
    elif final_relevant == initial_relevant:
        print(f"= Sin cambio en documentos relevantes")
    else:
        print(f"⚠ Disminución: {final_relevant - initial_relevant} documentos relevantes")

# Mostrar resultados para todas las consultas
print("\n" + "="*100)
print("SIMULACIÓN DE CONSULTAS - RESULTADOS DETALLADOS")
print("="*100)

for query_id in queries_df['query_id']:
    display_results(query_id, n_show=5)


SIMULACIÓN DE CONSULTAS - RESULTADOS DETALLADOS

CONSULTA: Q001
Título: Deep learning convolutional neural networks
Descripción: Papers about deep learning using convolutional neural networks for image processing and computer vision

────────────────────────────────────────────────────────────────────────────────────────────────────
RESULTADOS INICIALES (FAISS) - Top 5
────────────────────────────────────────────────────────────────────────────────────────────────────

1. [0705.2011] Score: 0.8795
   Título: Multi-Dimensional Recurrent Neural Networks
   Categorías: cs.AI cs.CV
   Abstract: Recurrent neural networks (RNNs) have proved effective at one dimensional sequence learning tasks, such as speech and online handwriting recognition. Some of the properties that make RNNs suitable for...
   URL: https://arxiv.org/abs/0705.2011

2. [0709.3642] Score: 0.8736
   Título: Functional Multi-Layer Perceptron: a Nonlinear Tool for Functional Data  Analysis
   Categorías: cs.NE
   Abstract: 

# 7. EVALUACIÓN DEL SISTEMA
Explicación: Evaluaremos el desempeño del sistema con métricas estándar:

Precision@k: % de documentos relevantes entre los top-k recuperados

Recall@k: % de documentos relevantes totales que están en top-k

Compararemos inicial vs re-ranking para medir la mejora

In [23]:
def calculate_precision_at_k(retrieved_docs, relevant_docs, k):
    """
    Precision@k = (# docs relevantes en top-k) / k
    """
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = len(set(retrieved_k) & set(relevant_docs))
    return relevant_retrieved / k if k > 0 else 0

def calculate_recall_at_k(retrieved_docs, relevant_docs, k):
    """
    Recall@k = (# docs relevantes en top-k) / (# total docs relevantes)
    """
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = len(set(retrieved_k) & set(relevant_docs))
    total_relevant = len(relevant_docs)
    return relevant_retrieved / total_relevant if total_relevant > 0 else 0

def evaluate_system(results, qrels_df, k_values=[5, 10, 20]):
    """
    Evalúa el sistema calculando métricas para todos los k especificados
    """
    metrics = defaultdict(list)
    query_metrics = {}

    for query_id, result in results.items():
        # Obtener documentos relevantes
        relevant_docs = qrels_df[
            (qrels_df['query_id'] == query_id) &
            (qrels_df['relevance'] > 0)
        ]['doc_id'].tolist()

        if len(relevant_docs) == 0:
            continue

        retrieved_docs = result['doc_ids']
        query_metrics[query_id] = {}

        for k in k_values:
            precision = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
            recall = calculate_recall_at_k(retrieved_docs, relevant_docs, k)

            metrics[f'P@{k}'].append(precision)
            metrics[f'R@{k}'].append(recall)

            query_metrics[query_id][f'P@{k}'] = precision
            query_metrics[query_id][f'R@{k}'] = recall

    # Calcular promedios
    avg_metrics = {metric: np.mean(values) for metric, values in metrics.items()}

    return avg_metrics, query_metrics

print("\n" + "="*100)
print("EVALUACIÓN DEL SISTEMA")
print("="*100)

# Evaluar recuperación inicial
print("\nEvaluando RECUPERACIÓN INICIAL (FAISS)...")
initial_metrics, initial_query_metrics = evaluate_system(initial_results, qrels_df, k_values=[5, 10, 20])

print(f"\n{'─'*50}")
print("MÉTRICAS - RECUPERACIÓN INICIAL")
print(f"{'─'*50}")
for metric in sorted(initial_metrics.keys()):
    print(f"{metric:8s}: {initial_metrics[metric]:.4f}")

# Evaluar re-ranking
print("\n\nEvaluando RESULTADOS CON RE-RANKING...")
final_metrics, final_query_metrics = evaluate_system(final_results, qrels_df, k_values=[5, 10, 20])

print(f"\n{'─'*50}")
print("MÉTRICAS - DESPUÉS DE RE-RANKING")
print(f"{'─'*50}")
for metric in sorted(final_metrics.keys()):
    print(f"{metric:8s}: {final_metrics[metric]:.4f}")


EVALUACIÓN DEL SISTEMA

Evaluando RECUPERACIÓN INICIAL (FAISS)...

──────────────────────────────────────────────────
MÉTRICAS - RECUPERACIÓN INICIAL
──────────────────────────────────────────────────
P@10    : 0.2100
P@20    : 0.2050
P@5     : 0.2400
R@10    : 0.0440
R@20    : 0.0701
R@5     : 0.0239


Evaluando RESULTADOS CON RE-RANKING...

──────────────────────────────────────────────────
MÉTRICAS - DESPUÉS DE RE-RANKING
──────────────────────────────────────────────────
P@10    : 0.2700
P@20    : 0.2300
P@5     : 0.3000
R@10    : 0.0465
R@20    : 0.0779
R@5     : 0.0255


In [24]:
# Métricas por consulta
print("\n" + "="*100)
print("MÉTRICAS POR CONSULTA")
print("="*100)

metrics_comparison = []

for query_id in queries_df['query_id']:
    query_title = queries_df[queries_df['query_id'] == query_id].iloc[0]['title']

    print(f"\n{query_id}: {query_title}")
    print(f"{'─'*80}")

    if query_id in initial_query_metrics and query_id in final_query_metrics:
        print(f"{'Métrica':<10} {'Inicial':<12} {'Re-ranking':<12} {'Mejora':<10}")
        print(f"{'-'*44}")

        for k in [5, 10, 20]:
            for metric_type in ['P', 'R']:
                metric_name = f'{metric_type}@{k}'
                initial_val = initial_query_metrics[query_id].get(metric_name, 0)
                final_val = final_query_metrics[query_id].get(metric_name, 0)
                improvement = final_val - initial_val

                print(f"{metric_name:<10} {initial_val:<12.4f} {final_val:<12.4f} {improvement:+.4f}")

                metrics_comparison.append({
                    'query_id': query_id,
                    'metric': metric_name,
                    'initial': initial_val,
                    'final': final_val,
                    'improvement': improvement
                })

# Convertir a DataFrame para análisis
metrics_comp_df = pd.DataFrame(metrics_comparison)


MÉTRICAS POR CONSULTA

Q001: Deep learning convolutional neural networks
────────────────────────────────────────────────────────────────────────────────
Métrica    Inicial      Re-ranking   Mejora    
--------------------------------------------
P@5        0.4000       0.6000       +0.2000
R@5        0.0400       0.0600       +0.0200
P@10       0.2000       0.4000       +0.2000
R@10       0.0400       0.0800       +0.0400
P@20       0.2500       0.2500       +0.0000
R@20       0.1000       0.1000       +0.0000

Q002: Natural language processing transformer models
────────────────────────────────────────────────────────────────────────────────
Métrica    Inicial      Re-ranking   Mejora    
--------------------------------------------
P@5        0.0000       0.4000       +0.4000
R@5        0.0000       0.0488       +0.0488
P@10       0.1000       0.3000       +0.2000
R@10       0.0244       0.0732       +0.0488
P@20       0.1500       0.2000       +0.0500
R@20       0.0732       0.097

# 8. ANÁLISIS DE RESULTADOS

Análisis de mejoras del re-ranking

Identificación de consultas con mayor/menor mejora

In [26]:
# Análisis detallado de mejoras
print("\n" + "="*100)
print("ANÁLISIS DETALLADO DE MEJORAS")
print("="*100)

print(f"\n{'Métrica':<10} {'Inicial':<12} {'Re-ranking':<12} {'Diferencia':<12} {'Mejora %':<10}")
print(f"{'─'*70}")

for metric in metrics_names:
    initial = initial_metrics.get(metric, 0)
    final = final_metrics.get(metric, 0)
    diff = final - initial
    mejora_pct = (diff / initial * 100) if initial > 0 else 0

    symbol = "✓" if diff > 0 else ("=" if diff == 0 else "✗")

    print(f"{metric:<10} {initial:<12.4f} {final:<12.4f} {diff:<+12.4f} {mejora_pct:+8.2f}% {symbol}")

# Promedio de mejora
avg_improvement = np.mean([
    ((final_metrics.get(m, 0) - initial_metrics.get(m, 0)) / initial_metrics.get(m, 1) * 100)
    for m in metrics_names if initial_metrics.get(m, 0) > 0
])

print(f"\n{'─'*70}")
print(f"Mejora promedio: {avg_improvement:+.2f}%")


ANÁLISIS DETALLADO DE MEJORAS

Métrica    Inicial      Re-ranking   Diferencia   Mejora %  
──────────────────────────────────────────────────────────────────────
P@5        0.2400       0.3000       +0.0600        +25.00% ✓
P@10       0.2100       0.2700       +0.0600        +28.57% ✓
P@20       0.2050       0.2300       +0.0250        +12.20% ✓
R@5        0.0239       0.0255       +0.0016         +6.76% ✓
R@10       0.0440       0.0465       +0.0025         +5.70% ✓
R@20       0.0701       0.0779       +0.0077        +11.02% ✓

──────────────────────────────────────────────────────────────────────
Mejora promedio: +14.87%


In [34]:



total_docs = len(docs_df)
total_queries = len(queries_df)
total_qrels = len(qrels_df)
avg_relevant_per_query = total_qrels / total_queries if total_queries > 0 else 0

# Calcular mejora promedio en P@10
p10_initial = initial_metrics.get('P@10', 0)
p10_final = final_metrics.get('P@10', 0)
p10_improvement = ((p10_final - p10_initial) / p10_initial * 100) if p10_initial > 0 else 0

print(f"""


Dataset:                    arXiv Scientific Papers
Papers procesados:          {total_docs:,}
Consultas evaluadas:        {total_queries}
Qrels generados:            {total_qrels:,}
Relevantes por consulta:    {avg_relevant_per_query:.1f} (promedio)

Modelos utilizados:
  • Embeddings:             {embedding_model.get_sentence_embedding_dimension()}D - allenai-specter / all-MiniLM-L6-v2
  • Re-ranking:             cross-encoder/ms-marco-MiniLM-L-6-v2

Parámetros:
  • Top-k inicial (FAISS):  {k_initial}
  • Top-k final:            {top_k_final}
  • Batch size:             {batch_size}

{'─'*100}
MÉTRICAS DE RENDIMIENTO
{'─'*100}

Recuperación Inicial (FAISS):
  Precision@5:              {initial_metrics.get('P@5', 0):.4f}
  Precision@10:             {initial_metrics.get('P@10', 0):.4f}
  Precision@20:             {initial_metrics.get('P@20', 0):.4f}
  Recall@10:                {initial_metrics.get('R@10', 0):.4f}

Después de Re-ranking:
  Precision@5:              {final_metrics.get('P@5', 0):.4f}  ({((final_metrics.get('P@5', 0) - initial_metrics.get('P@5', 0)) / initial_metrics.get('P@5', 1) * 100):+.1f}%)
  Precision@10:             {final_metrics.get('P@10', 0):.4f}  ({p10_improvement:+.1f}%)
  Precision@20:             {final_metrics.get('P@20', 0):.4f}  ({((final_metrics.get('P@20', 0) - initial_metrics.get('P@20', 0)) / initial_metrics.get('P@20', 1) * 100):+.1f}%)
  Recall@10:                {final_metrics.get('R@10', 0):.4f}  ({((final_metrics.get('R@10', 0) - initial_metrics.get('R@10', 0)) / initial_metrics.get('R@10', 1) * 100):+.1f}%)

Mejora promedio general:    {avg_improvement:+.2f}%

{'─'*100}
CONCLUSIONES
{'─'*100}

✓ FORTALEZAS DEL SISTEMA:
  1. Búsqueda semántica eficiente con FAISS permite procesar miles de documentos
  2. Embeddings especializados (SPECTER) capturan contexto científico
  3. Re-ranking con Cross-Encoder mejora significativamente la precisión
  4. Sistema escalable para millones de papers

✓ IMPACTO DEL RE-RANKING:
  • Mejora consistente en Precision@k (especialmente k pequeños)
  • Documentos más relevantes en primeras posiciones
  • Balance óptimo entre eficiencia (FAISS) y calidad (Cross-Encoder)

✓ APLICACIONES POTENCIALES:
  • Motor de búsqueda académica
  • Sistema de recomendación de papers
  • Análisis de tendencias en investigación
  • Identificación de trabajos relacionados
  • Construcción de grafos de conocimiento científico

⚠ LIMITACIONES Y MEJORAS FUTURAS:
  1. Qrels automáticos (idealmente usar juicios humanos)
  2. Expandir a más categorías científicas
  3. Integrar citaciones y métricas de impacto
  4. Implementar búsqueda multi-modal (texto + ecuaciones + figuras)
  5. Fine-tuning de modelos con datos específicos de arXiv




""")




Dataset:                    arXiv Scientific Papers
Papers procesados:          29,957
Consultas evaluadas:        10
Qrels generados:            1,743
Relevantes por consulta:    174.3 (promedio)

Modelos utilizados:
  • Embeddings:             768D - allenai-specter / all-MiniLM-L6-v2
  • Re-ranking:             cross-encoder/ms-marco-MiniLM-L-6-v2
  
Parámetros:
  • Top-k inicial (FAISS):  100
  • Top-k final:            20
  • Batch size:             32

────────────────────────────────────────────────────────────────────────────────────────────────────
MÉTRICAS DE RENDIMIENTO
────────────────────────────────────────────────────────────────────────────────────────────────────

Recuperación Inicial (FAISS):
  Precision@5:              0.2400
  Precision@10:             0.2100
  Precision@20:             0.2050
  Recall@10:                0.0440

Después de Re-ranking:
  Precision@5:              0.3000  (+25.0%)
  Precision@10:             0.2700  (+28.6%)
  Precision@20:        